In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, LSTM, Bidirectional, Dropout, Activation, BatchNormalization,\
GlobalMaxPooling1D, Reshape, TextVectorization, Embedding, GRU, SimpleRNN, Concatenate, Dot, Lambda, RepeatVector, Softmax, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16 as PretrainedModel, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

In [ ]:
# every image has 5 captions
!head -n 11 captions.txt

In [ ]:
images = []
captions = []

with open("captions.txt", 'r') as f:
    next(f)
    for line in f:
        image_caption = line.strip().split(",", 1)
        if len(image_caption) == 2:
            i, c = image_caption
            if not images or images[-1] != i:
                images.append(i)
            captions.append(c)

rand = np.random.randint(0, len(images))
sample_image = images[rand]
print(captions[rand*5:rand*5+5])
plt.imshow(image.load_img(f'Images/{sample_image}'))
plt.show()

In [ ]:
print(len(images), len(captions))

In [ ]:
IMAGE_SIZE = [200, 200]

In [ ]:
ptm = PretrainedModel(
    input_shape = IMAGE_SIZE + [3],
    weights = 'imagenet',
    include_top = False
)

In [ ]:
ptm.trainable = False

In [ ]:
print(ptm.summary())

In [ ]:
# output shape is 6*6*512
# should reshape for LSTM to 36*512
x = Reshape((-1, 512))(ptm.output)

In [ ]:
# create a model object
model = Model(inputs=ptm.input, outputs=x)

In [ ]:
print(model.summary())

In [ ]:
image_features = []

for image_id in images:
  image = load_img(f"Images/{image_id}", target_size=IMAGE_SIZE)
  image = img_to_array(image) / 255.0
  image = np.expand_dims(image, axis=0)

  features = model.predict(image)
  image_features.append(features.squeeze(axis=0))

In [ ]:
print(image_features[rand].shape)

In [ ]:
start_token = "<start>"
end_token = "<end>"

for i in range(len(captions)):
    captions[i] = captions[i].lower()
    captions[i] = re.sub(r"[^a-z\s]", "", captions[i])
    captions[i] = f"{start_token} {captions[i]} {end_token}"

In [ ]:
print(captions[rand*5])

In [ ]:
MAX_VOCAB_SIZE = 20_000

In [ ]:
vectorization_layer = TextVectorization(
    max_tokens=MAX_VOCAB_SIZE
)
vectorization_layer.adapt(captions)
vectorized_captions = vectorization_layer(captions).numpy()

In [ ]:
count = 0
for c in vectorized_captions:
    if len(c) != 37:
        count += 1
print(count)

In [ ]:
print(vectorized_captions[rand*5])
print(len(vectorized_captions[rand*5]))

In [ ]:
def data_generator(image_features, vectorized_captions, batch_size):
    while True:
        X_image_batch = []
        X_caption_batch = []
        y_batch = []

        for i, features in enumerate(image_features):
            cap_of_img = vectorized_captions[i*5:i*5+5]

            for cap in cap_of_img:
                for t in range(1, len(cap)):
                    input_caption = cap[:t]
                    target = cap[t]

                    X_image_batch.append(features)
                    X_caption_batch.append(input_caption)
                    y_batch.append(target)

                    if len(X_image_batch) == batch_size:
                        X_caption_batch = tf.keras.preprocessing.sequence.pad_sequences(
                            X_caption_batch, maxlen=37, padding='post'
                        )
                        yield {
                            'image_input': np.array(X_image_batch),
                            'caption_input': np.array(X_caption_batch)
                        }, np.array(y_batch)
                        X_image_batch, X_caption_batch, y_batch = [], [], []

In [ ]:
# Model definition with explicit layer names
img_i = Input(shape=(36, 512), name='image_input')
cap_i = Input(shape=(37,), name='caption_input', dtype='int32')

# encoder
img_x = GlobalMaxPooling1D()(img_i) # (N, 36, 512) -> (N, 512)
img_x = Dense(256, activation='relu')(img_x) # (N, 512) -> (N, 256)
img_x = BatchNormalization()(img_x) # (N, 256) -> (N, 256)
img_x = Dropout(0.2)(img_x) # (N, 256) -> (N, 256)

# embedding (masking tells model to ignore the padding)
cap_x = Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=256, mask_zero=True)(cap_i) # (N, 37) -> (N, 37, 256)
cap_x = Dropout(0.2)(cap_x) # (N, 37, 256) -> (N, 37, 256)
cap_x = Bidirectional(LSTM(256, return_sequences=True))(cap_x) # (N, 37, 256) -> (N, 37, 512)
cap_x = BatchNormalization()(cap_x) # (N, 37, 512) -> (N, 37, 512)
cap_x = Dropout(0.2)(cap_x) # (N, 37, 512) -> (N, 37, 512)

# attention
attention_x = Dense(128, activation='tanh')(cap_x) # (N, 37, 512) -> (N, 37, 128)
attention_x = BatchNormalization()(attention_x) # (N, 37, 128) -> (N, 37, 128)
attention_scores = Dense(1, activation='tanh')(attention_x) # (N, 37, 128) -> (N, 37, 1)
attention_scores = Reshape((37,))(attention_scores) # (N, 37, 1) -> (N, 37)
attention_weights = Dense(1, activation='softmax')(attention_scores) # (N, 37) -> (N, 1)
weighted_sum = Multiply()([cap_x, attention_weights]) # (N, 37, 512) * (N, 1) -> (N, 37, 512)
cap_x = GlobalMaxPooling1D()(weighted_sum) # (N, 37, 512) -> (N, 512)

# combine
x = Concatenate()([img_x, cap_x]) # (N, 256) and (N, 512) -> (N, 768)
x = Dense(512, activation='relu')(x) # (N, 768) -> (N, 512)
x = BatchNormalization()(x) # (N, 512) -> (N, 512)
x = Dropout(0.2)(x) # (N, 512) -> (N, 512)
x = Dense(MAX_VOCAB_SIZE, activation='softmax')(x) # (N, 512) -> (N, MAX_VOCAB_SIZE)

In [ ]:
model = Model(inputs=[img_i, cap_i], outputs=x)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
print(model.summary())

In [ ]:
split_index = len(image_features) * 4 // 5

train_features = image_features[:split_index]
val_features = image_features[split_index:]

train_captions = vectorized_captions[:split_index * 5]
val_captions = vectorized_captions[split_index * 5:]

In [ ]:
batch_size = 128

steps_per_epoch = len(train_features) * 5 // batch_size
validation_steps = len(val_features) * 5 // batch_size

# Create TensorFlow datasets with matching input names
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_features, train_captions, batch_size),
    output_signature=(
        {
            'image_input': tf.TensorSpec(shape=(batch_size, 36, 512), dtype=tf.float32),
            'caption_input': tf.TensorSpec(shape=(batch_size, 37), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(batch_size,), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(val_features, val_captions, batch_size),
    output_signature=(
        {
            'image_input': tf.TensorSpec(shape=(batch_size, 36, 512), dtype=tf.float32),
            'caption_input': tf.TensorSpec(shape=(batch_size, 37), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(batch_size,), dtype=tf.float32)
    )
)

r = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=val_dataset,
    validation_steps=validation_steps
)

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
model.save("captioning_model.h5")

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
import nltk
nltk.download('punkt')

In [ ]:
def generate_caption(model, image_features, vectorization_layer, max_length=37):
    # Initialize with start token
    caption = ['<start>']
    
    # Convert to sequence
    input_caption = vectorization_layer(caption).numpy()
    
    # Generate caption word by word
    for i in range(max_length):
        # Prepare inputs
        current_caption = tf.keras.preprocessing.sequence.pad_sequences(
            [input_caption], maxlen=37, padding='post'
        )
        
        # Make prediction
        predictions = model.predict(
            {
                'image_input': np.expand_dims(image_features, axis=0),
                'caption_input': current_caption
            }, 
            verbose=0
        )
        
        # Get predicted word index
        predicted_id = np.argmax(predictions[0])
        
        # Convert to word
        predicted_word = vectorization_layer.get_vocabulary()[predicted_id]
        
        # Break if end token
        if predicted_word == '<end>':
            break
            
        # Add to caption
        caption.append(predicted_word)
        input_caption = vectorization_layer(caption).numpy()
    
    return ' '.join(caption[1:])  # Remove start token

In [ ]:
def evaluate_bleu(model, val_features, val_captions, vectorization_layer):
    references = []
    hypotheses = []
    
    # Generate predictions for validation set
    for i, image_feature in enumerate(val_features):
        # Get reference captions for this image (5 captions per image)
        image_captions = val_captions[i*5:(i+1)*5]
        reference = []
        
        # Process reference captions
        for cap in image_captions:
            # Convert indices back to words and tokenize
            cap_words = [vectorization_layer.get_vocabulary()[idx] for idx in cap if idx != 0]
            # Remove start and end tokens
            cap_words = [word for word in cap_words if word not in ['<start>', '<end>', '']]
            reference.append(cap_words)
        
        references.append(reference)
        
        # Generate prediction
        predicted_caption = generate_caption(model, image_feature, vectorization_layer)
        # Tokenize prediction
        hypothesis = predicted_caption.split()
        # Remove start and end tokens if present
        hypothesis = [word for word in hypothesis if word not in ['<start>', '<end>', '']]
        
        hypotheses.append(hypothesis)
        
        # Print progress
        if (i+1) % 100 == 0:
            print(f"Processed {i+1}/{len(val_features)} images")
    
    # Calculate BLEU scores
    bleu1 = corpus_bleu(references, hypotheses, weights=(1.0, 0, 0, 0))
    bleu2 = corpus_bleu(references, hypotheses, weights=(0.5, 0.5, 0, 0))
    bleu3 = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0))
    bleu4 = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))
    
    return {
        'bleu1': bleu1,
        'bleu2': bleu2,
        'bleu3': bleu3,
        'bleu4': bleu4
    }

In [ ]:
bleu_scores = evaluate_bleu(model, val_features, val_captions, vectorization_layer)

print("BLEU Scores:")
for metric, score in bleu_scores.items():
    print(f"{metric}: {score:.4f}")